<a href="https://colab.research.google.com/github/santiagorey16/PROGCOM-A/blob/main/mejora_de_proyecto_de_carrerra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import tkinter as tk
from tkinter import ttk, messagebox

def f(t, y):
    return 3 * y + 2


def solucion_exacta(t, y0):
    """
    dy/dt = 3y + 2  →  y(t) = -2/3 + C * e^{3t}
    Con y(0) = y0  →  C = y0 + 2/3
    """
    C = y0 + 2.0 / 3.0
    return -2.0 / 3.0 + C * math.exp(3.0 * t)


def euler(t0, tf, y0, h):
    n_pasos = int((tf - t0) / h) + 1
    t_values = [t0 + i * h for i in range(n_pasos)]
    y_values = [0.0] * n_pasos
    y_values[0] = y0

    for i in range(1, n_pasos):
        y_values[i] = y_values[i - 1] + h * f(t_values[i - 1], y_values[i - 1])

    return t_values, y_values


class AppEDO(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("EDO: dy/dt = 3y + 2 (Euler, sin librerías externas)")
        self.geometry("950x600")

        # Layout general
        self.columnconfigure(0, weight=0)
        self.columnconfigure(1, weight=1)
        self.rowconfigure(0, weight=1)

        self._crear_panel_controles()
        self._crear_panel_grafica()

    # ----- Controles -----

    def _crear_panel_controles(self):
        frame = ttk.Frame(self, padding=10)
        frame.grid(row=0, column=0, sticky="ns")

        titulo = ttk.Label(
            frame,
            text="Parámetros de la EDO\n dy/dt = 3y + 2",
            font=("Segoe UI", 11, "bold")
        )
        titulo.pack(pady=(0, 10))

        self.entry_t0 = self._crear_campo(frame, "t₀ (inicio):", "0")
        self.entry_tf = self._crear_campo(frame, "t_f (final):", "2")
        self.entry_y0 = self._crear_campo(frame, "y₀ (condición inicial):", "1")
        self.entry_h  = self._crear_campo(frame, "h (paso):", "0.1")

        self.mostrar_exacta = tk.BooleanVar(value=True)
        chk = ttk.Checkbutton(
            frame,
            text="Mostrar solución exacta",
            variable=self.mostrar_exacta
        )
        chk.pack(pady=5)

        btn_resolver = ttk.Button(frame, text="Resolver y graficar",
                                  command=self.resolver_y_graficar)
        btn_resolver.pack(pady=10, fill="x")

        self.label_info = ttk.Label(
            frame,
            text="Tip:\nPrueba h = 0.5, 0.2, 0.05\npara ver el error de Euler.",
            justify="left"
        )
        self.label_info.pack(pady=10)

    def _crear_campo(self, parent, texto, valor_inicial):
        cont = ttk.Frame(parent)
        cont.pack(fill="x", pady=3)

        lbl = ttk.Label(cont, text=texto)
        lbl.pack(side="left")

        entry = ttk.Entry(cont, width=10)
        entry.insert(0, valor_inicial)
        entry.pack(side="right")

        return entry

    # ----- Panel de gráfica (Canvas) -----

    def _crear_panel_grafica(self):
        frame = ttk.Frame(self, padding=10)
        frame.grid(row=0, column=1, sticky="nsew")
        frame.rowconfigure(0, weight=1)
        frame.columnconfigure(0, weight=1)

        self.canvas_width = 750
        self.canvas_height = 520

        self.canvas = tk.Canvas(
            frame,
            width=self.canvas_width,
            height=self.canvas_height,
            bg="white"
        )
        self.canvas.grid(row=0, column=0, sticky="nsew")

        # Dibujar mensaje inicial
        self.canvas.create_text(
            self.canvas_width // 2,
            self.canvas_height // 2,
            text="Haz clic en 'Resolver y graficar'\npara ver la solución.",
            font=("Segoe UI", 12),
            fill="gray"
        )

    # ----- Lógica de resolución y dibujo -----

    def resolver_y_graficar(self):
        # Leer parámetros
        try:
            t0 = float(self.entry_t0.get())
            tf = float(self.entry_tf.get())
            y0 = float(self.entry_y0.get())
            h  = float(self.entry_h.get())

            if h <= 0:
                raise ValueError("El paso h debe ser positivo.")
            if tf <= t0:
                raise ValueError("t_f debe ser mayor que t₀.")

        except ValueError as e:
            messagebox.showerror(
                "Error en parámetros",
                f"Revisa los valores ingresados.\n\nDetalle: {e}"
            )
            return

        # Solución numérica con Euler
        t_num, y_num = euler(t0, tf, y0, h)

        # Solución exacta (si se solicita)
        t_ex = []
        y_ex = []
        if self.mostrar_exacta.get():
            n_exact = 300
            dt = (tf - t0) / (n_exact - 1)
            for i in range(n_exact):
                t_val = t0 + i * dt
                t_ex.append(t_val)
                y_ex.append(solucion_exacta(t_val, y0))

        # Calcular rangos para escalar al Canvas
        t_min = t0
        t_max = tf

        y_min = min(y_num)
        y_max = max(y_num)

        if y_ex:
            y_min = min(y_min, min(y_ex))
            y_max = max(y_max, max(y_ex))

        if y_max == y_min:
            y_max += 1.0
            y_min -= 1.0

        # Limpiar canvas
        self.canvas.delete("all")

        # Margenes
        margin_left = 60
        margin_right = 20
        margin_top = 20
        margin_bottom = 60

        # Funciones para convertir (t, y) → (x, y_pix)
        def to_x(t):
            return margin_left + (t - t_min) / (t_max - t_min) * (
                self.canvas_width - margin_left - margin_right
            )

        def to_y(y):
            # y grande debe ir abajo, por eso se invierte
            return margin_top + (y_max - y) / (y_max - y_min) * (
                self.canvas_height - margin_top - margin_bottom
            )

        # Dibujar ejes
        # Eje t
        x0 = to_x(t_min)
        x1 = to_x(t_max)
        y_axis = to_y(0) if y_min <= 0 <= y_max else to_y(y_min)
        self.canvas.create_line(x0, y_axis, x1, y_axis, fill="black")

        # Eje y
        t_axis = 0 if t_min <= 0 <= t_max else t_min
        x_axis = to_x(t_axis)
        self.canvas.create_line(x_axis, to_y(y_min), x_axis, to_y(y_max), fill="black")

        # Etiquetas de ejes
        self.canvas.create_text(
            self.canvas_width // 2, self.canvas_height - 30,
            text="t", font=("Segoe UI", 11)
        )
        self.canvas.create_text(
            25, self.canvas_height // 2,
            text="y(t)", font=("Segoe UI", 11)
        )

        # Título
        self.canvas.create_text(
            self.canvas_width // 2, 15,
            text=f"dy/dt = 3y + 2   (Euler, h = {h})",
            font=("Segoe UI", 12, "bold")
        )

        # Dibujar solución exacta (línea roja)
        if y_ex:
            for i in range(1, len(t_ex)):
                x_prev = to_x(t_ex[i - 1])
                y_prev = to_y(y_ex[i - 1])
                x_curr = to_x(t_ex[i])
                y_curr = to_y(y_ex[i])
                self.canvas.create_line(x_prev, y_prev, x_curr, y_curr, fill="red")
            self.canvas.create_text(
                margin_left + 10, margin_top + 15,
                text="Exacta",
                fill="red",
                font=("Segoe UI", 10, "bold")
            )

        # Dibujar solución de Euler (línea azul con puntos)
        for i in range(1, len(t_num)):
            x_prev = to_x(t_num[i - 1])
            y_prev = to_y(y_num[i - 1])
            x_curr = to_x(t_num[i])
            y_curr = to_y(y_num[i])
            self.canvas.create_line(x_prev, y_prev, x_curr, y_curr, fill="blue")
            self.canvas.create_oval(
                x_curr - 2, y_curr - 2, x_curr + 2, y_curr + 2,
                fill="blue", outline="blue"
            )

        self.canvas.create_text(
            margin_left + 10, margin_top + 30,
            text="Euler",
            fill="blue",
            font=("Segoe UI", 10, "bold")
        )


if __name__ == "__main__":
    print("Resolviendo la EDO: dy/dt = 3y + 2, y(0) = 1")
    app = AppEDO()
    app.mainloop()